In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

在Vertex AI上构建一个欺诈检测模型

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/fraud_detection/fraud-detection-model.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/fraud_detection/fraud-detection-model.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
   <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/workbench/fraud_detection/fraud-detection-model.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>

注意：此笔记本已在以下环境中进行测试：

- Python版本= 3.9

## 概述

本教程向您展示如何使用诸如scikit-learn、Vertex AI和[What-IF工具(WIT)](https://cloud.google.com/ai-platform/prediction/docs/using-what-if-tool)等工具构建、部署和分析简单的[随机森林](https://en.wikipedia.org/wiki/Random_forest)模型预测合成欺诈交易数据集来解决金融欺诈检测问题。

**注意：** 本笔记本中使用的What-If工具小部件仅在Colab环境中运行。它不是Vertex AI用户管理的笔记本实例的显式支持。

了解更多关于[Vertex AI Workbench](https://cloud.google.com/vertex-ai/docs/workbench/introduction)和[定制训练](https://cloud.google.com/vertex-ai/docs/training/custom-training)。

### 目标

本教程演示了如何使用一个合成的金融数据集进行数据分析和建模。模型训练的目标是识别交易中的欺诈案例。然后，训练好的模型将部署到 Vertex AI Endpoint，并使用 What-If 工具进行分析。本教程中的步骤如下：

本教程使用以下 Google Cloud ML 服务和资源：

- Vertex AI 模型
- Vertex AI Endpoint

操作步骤包括：

- 安装所需的库
- 从 Cloud Storage 存储桶中读取数据集
- 对数据集进行探索性分析
- 对数据集进行预处理
- 使用 scikit-learn 训练一个随机森林模型
- 将模型保存到 Cloud Storage 存储桶
- 创建一个 Vertex AI 模型资源并部署到一个端点
- 在测试数据上运行 What-If 工具
- 卸载模型并清理模型资源

### 数据集

本教程中使用的数据集可在Kaggle上公开获取。请参阅[用于欺诈检测的合成金融数据集](https://www.kaggle.com/ealaxi/paysim1)。

### 成本


本教程使用的是 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage
定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/)
根据您预期的使用情况生成成本估算。

## 安装

安装以下必需的软件包以执行此笔记本。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                witwidget \
                                fsspec \
                                gcsfs
! pip3 install --quiet scikit-learn==1.2 \
                        protobuf==3.20.1

### 仅限于Colab：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置您的谷歌云项目

**无论您使用什么笔记本环境，以下步骤都是必不可少的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用额用于计算/存储成本。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

4. 如果您在本地运行此笔记本，请确保安装了[Cloud SDK](https://cloud.google.com/sdk)。

#### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下方法：
- 运行 `gcloud config list`
- 运行 `gcloud projects list`
- 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# set the project id
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。
了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的 Google Cloud 帐户

根据您的 Jupyter 环境，您可能需要手动进行身份验证。请按照下面的相关说明进行操作。

**1. Vertex AI Workbench**
- 不需要进行任何操作，因为您已经验证过了。

**2. 本地 JupyterLab 实例**，请取消注释并运行。

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账号或其他
* 请参考如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples为您的服务帐号授予Cloud Storage权限。

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶尚不存在时，才可以运行以下单元格来创建您的云存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

导入库

In [ ]:
import os
import pickle
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from google.cloud import aiplatform, storage
from IPython.display import display
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (average_precision_score, classification_report,
                             confusion_matrix, f1_score)
from sklearn.model_selection import train_test_split
from witwidget.notebook.visualization import WitConfigBuilder, WitWidget

warnings.filterwarnings("ignore")

### 初始化 Python 的 Vertex AI SDK

为您的项目初始化 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## 加载数据集

使用Pandas从公共csv文件路径加载数据集。

In [ ]:
# set the dataset path
DATASET_SOURCE_PATH = "gs://cloud-samples-data/vertex-ai/managed_notebooks/fraud_detection/fraud_detection_data.csv"
# read the csv data using pandas
df = pd.read_csv(DATASET_SOURCE_PATH)


## 分析数据集
快速查看数据集和行数。

In [ ]:
# print the shape of dataframe
print("shape : ", df.shape)
# display the dataframe
df.head()

检查空值。

In [ ]:
# print the total null count per column
df.isnull().sum()

检查涉及的交易类型和每种类型所关联的总金额。

In [ ]:
# check value counts for type
print(df.type.value_counts())
# show total amount per type as a bar chart
var = df[["type", "amount"]].groupby("type").sum()
var.plot(kind="bar")
plt.title("Total amount per transaction type")
plt.xlabel("Type of Transaction")
plt.ylabel("Amount")
plt.show()

## 处理不平衡数据

尽管结果变量“isFraud”看起来非常不平衡，但可以在其上训练一个基本模型来检查数据中欺诈交易的质量。如有必要，可以考虑采用欠取样多数类或过取样少数类等对策。

In [ ]:
# Count number of fraudulent/non-fraudulent transactions
df.isFraud.value_counts()

展示以饼图形式显示检测到的欺诈交易百分比。

In [ ]:
# plot the percentage of frauds as a pie chart
piedata = df[["isFlaggedFraud", "isFraud"]].groupby(["isFlaggedFraud"]).sum()
f, axes = plt.subplots(1, 1, figsize=(6, 6))
axes.set_title("% of fraud transaction detected")
piedata.plot(
    kind="pie", y="isFraud", ax=axes, fontsize=14, shadow=False, autopct="%1.1f%%"
)
axes.set_ylabel("")
plt.legend(loc="upper left", labels=["Not Detected", "Detected"])
plt.show()

准备建模数据
为了准备用于训练的数据集，需要删除一些包含唯一数据（'nameOrig'，'nameDest'）和冗余字段（'isFlaggedFraud'）的列。描述交易类型并对欺诈检测很重要的分类字段“type”需要进行独热编码。

In [ ]:
# drop the unnecessary fields
df.drop(["nameOrig", "nameDest", "isFlaggedFraud"], axis=1, inplace=True)
# encode the "type" field
X = pd.concat([df.drop("type", axis=1), pd.get_dummies(df["type"])], axis=1)
X.head()

将结果变量从训练数据中移除。

In [ ]:
# copy the target data
y = X[["isFraud"]]
# remove the target field from the features
X = X.drop(["isFraud"], axis=1)

将数据分割并将70％用于训练，30％用于测试。

在分割时，您可以为Sklearn的`train_test_split`方法指定以下参数：

- `*arrays`：特征数组（X）和目标数组（y）。
- `test_size`：测试样本的百分比（浮点数）或数量（整数）。
- `random_state`：在应用拆分之前控制应用于数据的洗牌。传递一个整数以使多个函数调用产生可重现的输出。
- `stratify`：如果没有，则不执行分层抽样。

由于数据不平衡，您在分割时使用分层抽样。了解更多关于[分层抽样和用于训练-测试拆分的其他参数](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)。

In [ ]:
# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
# check the data shapes
print(X_train.shape, X_test.shape)

## 训练一个随机森林模型

在预处理后的训练数据集上训练一个简单的随机森林分类器。

注意：在定义`RandomForestClassifier`对象时，将`n_jobs`设置为-1可以利用所有处理器并行化训练过程。

了解更多关于[随机森林算法](https://en.wikipedia.org/wiki/Random_forest)和Sklearn的[RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)。

In [ ]:
# create a randomforestclassifier object
forest = RandomForestClassifier(n_jobs=-1, verbose=1)
# fit the model on the data
forest.fit(X_train, y_train)

## 分析结果

生成测试数据上的预测类别和概率分数。

使用以下指标评估模型：

- `AP`：平均精度将精度-召回曲线总结为在每个阈值处达到的精度的加权平均值，前一个阈值的召回率增加用作权重。
- F1分数：F1分数是精度和召回率的调和平均值。
- 混淆矩阵：显示准确预测的真正例、真负例、假正例和假负例的矩阵。
- 分类报告：Sklearn的分类报告是一个文本报告，显示主要的分类指标，如精度、召回率、F1分数、准确性，以及这些指标的加权平均值和宏平均值。

了解更多关于[Sklearn指标](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics)。

In [ ]:
# predict probability of fraudulent transactions over test set
y_prob = forest.predict_proba(X_test)
# predict the classes over test set
y_pred = forest.predict(X_test)
# check the average precision score
print("AP :", (average_precision_score(y_test, y_prob[:, 1])))
# check the f1-score
print("F1 - score :", (f1_score(y_test, y_pred)))
# print the confusion matrix
print("Confusion_matrix : ")
print(confusion_matrix(y_test, y_pred))
# print the classification report
print("classification_report")
print(classification_report(y_test, y_pred))

使用`RandomForestClassifier`的`feature_importances_`函数来更好地了解哪些特征对模型最有用。

In [ ]:
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
forest_importances = pd.Series(importances, index=list(X_train))
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature Importance for Fraud Transaction Detection Model")
ax.set_ylabel("Importance")
fig.tight_layout()

将模型保存到云存储

将您的模型保存到一个pickle文件中，然后将您的模型上传到云存储存储桶中。上传的模型路径稍后用于在Vertex AI模型注册表中创建一个模型。

注意：您也可以使用最新的Python版Vertex AI SDK从您的本地环境将模型上传到Vertex AI模型注册表。

In [ ]:
# save the trained model to a local file
LOCAL_FILE_NAME = "model.pkl"
with open(LOCAL_FILE_NAME, "wb") as file:
    pickle.dump(forest, file)

# Upload the saved model file to Cloud Storage
BLOB_PATH = "fraud-detect-model-path-unique"  # @param {type:"string"}
BLOB_NAME = os.path.join(BLOB_PATH, LOCAL_FILE_NAME)

bucket = storage.Client(PROJECT_ID).bucket(BUCKET_URI[5:])
blob = bucket.blob(BLOB_NAME)
blob.upload_from_filename(LOCAL_FILE_NAME)

在Vertex AI中创建一个模型

在Vertex AI模型注册表中设置模型创建所需的参数。

In [ ]:
# set model display name
MODEL_DISPLAY_NAME = "fraud-detection-model-unique"  # @param {type:"string"}
# set the GCS path to the model artifact
ARTIFACT_GCS_PATH = f"{BUCKET_URI}/{BLOB_PATH}"
# set the prediction container uri
SERVING_CONTAINER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-2:latest"
)

使用`Model.upload`方法在Vertex AI中创建一个模型资源。

了解有关[Vertex AI模型注册表](https://cloud.google.com/vertex-ai/docs/model-registry/introduction)的更多信息。

In [ ]:
# create a Vertex AI model resource
model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=ARTIFACT_GCS_PATH,
    serving_container_image_uri=SERVING_CONTAINER_IMAGE_URI,
)
# print the model's display name
print("Display name:\n", model.display_name)
# print the model's resource name
print("Resource name:\n", model.resource_name)

## 创建端点

设置显示名称并创建用于部署模型的端点。

In [ ]:
# set the endpoint display name
ENDPOINT_DISPLAY_NAME = "fraud-detect-endpoint-unique"  # @param {type:"string"}
# create the Endpoint
endpoint = aiplatform.Endpoint.create(display_name=ENDPOINT_DISPLAY_NAME)
# print the endpoint display name
print("Display name:\n", endpoint.display_name)
# print the endpoint resource name
print("Resource name:\n", endpoint.resource_name)

### 部署模型到端点

为端点部署设置以下参数：

- `endpoint`：在上一步创建的 Vertex AI 端点资源。
- `deployed_model_display_name`：模型的显示名称。如果未提供，则使用模型的显示名称。
- `machine_type`：用于在端点上提供模型所需的机器类型。

In [ ]:
# set the display name for the deployed model
DEPLOYED_MODEL_NAME = "fraud-detection-deployed-model"
# set the machine type for the endpoint
MACHINE_TYPE = "n1-standard-2"

部署模型到创建的端点。

In [ ]:
# deploy the model to the endpoint
model.deploy(
    endpoint=endpoint,
    deployed_model_display_name=DEPLOYED_MODEL_NAME,
    machine_type=MACHINE_TYPE,
)
# print the model display name
print(model.display_name)
# print the model resource name
print(model.resource_name)

## 什么是工具

什么是工具可用于分析测试数据上的模型预测。查看[什么是工具的简介](https://pair-code.github.io/what-if-tool/)。

在本教程中，将使用本地训练模型以及部署在Vertex AI端点上的模型配置和运行什么是工具。

[WitConfigBuilder](https://github.com/PAIR-code/what-if-tool/blob/master/witwidget/notebook/visualization.py#L30) 提供了 `set_ai_platform_model()` 方法，用于配置什么是工具，以使用部署为AI Platform模型版本的模型。此功能目前仅支持AI Platform，不支持Vertex AI模型。但是，还可以通过 `set_custom_predict_fn()` 方法传递一个自定义函数来生成预测，其中可以传递本地训练模型或从Vertex AI模型返回预测的函数。

### 准备测试样本

从测试数据中为两个可用的类别（欺诈/非欺诈）保留一些样本，以便使用“假设工具”来分析模型。

In [ ]:
# set sample size
SAMPLE = 10

# collect samples for each class-label from the test data
pos_samples = y_test[y_test["isFraud"] == 1].sample(SAMPLE).index
neg_samples = y_test[y_test["isFraud"] == 0].sample(SAMPLE).index
test_samples_y = pd.concat([y_test.loc[pos_samples], y_test.loc[neg_samples]])
test_samples_X = X_test.loc[test_samples_y.index].copy()

在本地模型上运行假设工具

使用本地模型的`predict_proba`函数构建假设工具小部件。

以下步骤生成一个用于分析预测的交互式小部件。

注意：此下一单元仅在支持假设工具的Colab环境中运行。

In [ ]:
# check for Colab environment
IS_COLAB = "google.colab" in sys.modules

# run what-if tool
if IS_COLAB:
    # define target and labels
    TARGET_FEATURE = "isFraud"
    LABEL_VOCAB = ["not-fraud", "fraud"]

    # define the function to adjust the predictions

    def adjust_prediction(pred):
        return [1 - pred, pred]

    # Combine the features and labels into one array for the What-If Tool
    test_examples = np.hstack(
        (test_samples_X.to_numpy(), test_samples_y.to_numpy().reshape(-1, 1))
    )

    # Configure the WIT to run on the locally trained model
    config_builder = (
        WitConfigBuilder(
            test_examples.tolist(), test_samples_X.columns.tolist() + ["isFraud"]
        )
        .set_custom_predict_fn(forest.predict_proba)
        .set_target_feature(TARGET_FEATURE)
        .set_label_vocab(LABEL_VOCAB)
    )

    # display the WIT widget
    display(WitWidget(config_builder, height=600))

### 在部署的Vertex AI模型上运行What-If工具

在这一步中，您需要定义一个函数，该函数将向终端部署的模型发送请求，并返回格式化的预测结果。然后使用该函数来构建What-IF工具小部件，以分析这些预测结果。

In [ ]:
# run the what-if tool
if IS_COLAB:
    # configure the target and class-labels
    TARGET_FEATURE = "isFraud"
    LABEL_VOCAB = ["not-fraud", "fraud"]

    # function to return predictions from the deployed Model

    def endpoint_predict_sample(instances: list):
        prediction = endpoint.predict(instances=instances)
        preds = [[1 - i, i] for i in prediction.predictions]
        return preds

    # Combine the features and labels into one array for the What-If Tool
    test_examples = np.hstack(
        (test_samples_X.to_numpy(), test_samples_y.to_numpy().reshape(-1, 1))
    )

    # Configure the WIT with the prediction function
    config_builder = (
        WitConfigBuilder(
            test_examples.tolist(), test_samples_X.columns.tolist() + ["isFraud"]
        )
        .set_custom_predict_fn(endpoint_predict_sample)
        .set_target_feature(TARGET_FEATURE)
        .set_label_vocab(LABEL_VOCAB)
    )

    # run the WIT-widget
    display(WitWidget(config_builder, height=400))

## 清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除您在教程中使用的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- Vertex AI 端点
- Vertex AI 模型
- Cloud Storage 存储桶

In [ ]:
# undeploy the model from the endpoint
endpoint.undeploy_all()

# delete the endpoint
endpoint.delete()

# delete the model
model.delete()

# delete the bucket
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI

# delete the local files
! rm $LOCAL_FILE_NAME